In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
spark

In [5]:
spark.conf.set("spark.sql.adpative.enabled", "false")
spark.conf.set("spark.sql.optimized.dynamicPartitionPruning.enabled", "true")
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

In [6]:
df = spark.read.format('csv')\
        .option('inferSchema', True)\
        .option('header', True)\
        .load('BigMart Sales.csv')

In [7]:
df.show(7)

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|
|          DRC01|       5.92|         Regular|    0.019278216|         Soft Drinks| 48.2692|           OUT018|                     2009|     Medium|              Tier 3|Superma

In [8]:
from pyspark.sql.functions import *

In [15]:
df.write.format("parquet")\
    .mode("append")\
    .partitionBy("Item_Identifier")\
    .save("dpp_partitioned")

#### Non Partitioned Data

In [10]:
df.write.format("parquet")\
    .mode("append")\
    .save("dpp_nonpartitioned")

#### Dataframes

In [11]:
df1 = spark.read.format("parquet")\
    .load("dpp_partitioned")

In [12]:
df2 = spark.read.format("parquet")\
    .load("dpp_nonpartitioned")

#### Joins

In [13]:
df_join = df1.join(df2.filter(col("Outlet_Type") == 'Grocery Store'), df2['Item_Identifier'] == df2['Item_Identifier'], "inner")

In [14]:
df_join.show()

+---------------+-----------+----------------+---------------+---------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-------------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|Item_Outlet_Sales|      Outlet_Type|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|  Outlet_Type|Item_Outlet_Sales|
+---------------+-----------+----------------+---------------+---------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+---------------+